**goal is to trim 15 510 geographical articles, to charaterize each headphrase by the Geographical Entities (GE) mentionned in the article body**

**Input :** `geobook_plain_23082024.pkl`

**Output :** `geobook_23082024.pkl`


In [4]:
import sys
sys.path.append('/home/antoine/Documents/GitHub/linkencyclo')
datapath = '/home/antoine/Documents/GitHub/datas/'
from stanza.models.common.doc import Span, Token

from EncycloObject import Article, Book
import pickle

We load a `Book` instance :

In [43]:
with open(datapath+'geobook_plain_23082024.pkl', 'rb') as f:
    plain_geobook = pickle.load(f)
print(plain_geobook)

Book with 15510 articles
plain version of the geographical articles
Attributes :
dict_keys(['volume', 'numero', 'authors', 'text', 'hash', 'artfl', 'gold_coords', 'enccre', 'article_id', 'gold_qid', 'headphrase'])


Example of the Article instance with hash `7/2925/GRENOBLE`

In [6]:
grenoble = mybook._reach_article(headphrase='grenoble')
grenoble.artfl, grenoble.hash, grenoble.text

('https://artflsrv04.uchicago.edu/philologic4.7/encyclopedie0922/navigate/7/2925',
 '7/2925/GRENOBLE',
 'GRENOBLE, Gratianopolis, (Géogr.) ancienne ville de France, capitale du Dauphiné, avec un évêché suffragant de Vienne, & un parlement érigé en 1493 par Louis XI. qui n\'étoit encore que dauphin ;  mais son pere ratifia cette érection deux ans après.\nGrenoble est sur l\'Isere, à onze lieues S O. de Chambéri, quarante-deux N. O. de Turin, seize S. E. de Vienne, cent vingt-quatre S. O. de Paris. Long. suivant Harris, 23d. 31\'. 15". suivant Cassini, 23d. 14\'. 15". latit 45d. 11\'.\nCette ville reçut le nom de Gratianopolis de l\'empereur  Gratien fils de Valentinien I. car elle s\'appelloit  auparavant Cularo ; & c\'est sous ce nom qu\'il en est parlé dans une lettre de Plancus à Cicéron, epist. xxiij. Long-tems après, les Romains l\'érigerent en cité: dans le cinquieme siecle, elle fut assujettie par les Bourguignons, & dans le sixieme par les François Mérovingiens ; ensuite elle a 

# Part-of-Speech parsing
Stanza doc are stored in the `.parsed` attribute

In [5]:
import stanza
nlpstanza = stanza.Pipeline(lang='fr', processors='tokenize,mwt, pos')

2024-08-23 11:44:31 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-08-23 11:44:31 INFO: Downloaded file to /home/antoine/stanza_resources/resources.json
2024-08-23 11:44:32 INFO: Loading these models for language: fr (French):
| Processor | Package         |
-------------------------------
| tokenize  | combined        |
| mwt       | combined        |
| pos       | combined_charlm |

2024-08-23 11:44:32 INFO: Using device: cpu
2024-08-23 11:44:32 INFO: Loading: tokenize
2024-08-23 11:44:32 INFO: Loading: mwt
2024-08-23 11:44:32 INFO: Loading: pos
2024-08-23 11:44:32 INFO: Done loading processors!


In [29]:
for article in plain_geobook:
    article.parsed = nlpstanza(article.text)
    #break

# Named Entity Recognition
We apply the BERT model to identify the GE mentionned in each article-body.

From
> 7/2925/GRENOBLE 
> 
> 'GRENOBLE, Gratianopolis, (Géogr.) ancienne ville de France, capitale du Dauphiné, avec un évêché suffragant de Vienne, ...'

We want to store in the `article.ner` attribute the NER tags like :
```python
[
    {'entity_group': 'NC_Spatial', 'score': 0.9601506, 'word': 'ancienne ville', 'start': 26, 'end': 40},
    {'entity_group': 'NP_Spatial', 'score': 0.96639144, 'word': 'France', 'start': 44, 'end': 50},
    {'entity_group': 'NC_Spatial', 'score': 0.96316826, 'word': 'capitale', 'start': 52, 'end': 60},
    {'entity_group': 'NP_Spatial', 'score': 0.9662918, 'word': 'Dauphiné', 'start': 64, 'end': 72},
    ...
]
```


In [30]:
from transformers import pipeline
bert = pipeline("token-classification", model="GEODE/bert-base-french-cased-edda-ner",
                aggregation_strategy="simple")

/home/antoine/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/antoine/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [31]:
for article in plain_geobook:
    article.ner = article._apply_pipeline(bert)
    #break

We then add NER tags to the Stanza doc thanks to the method `Article._enrich_stanzadoc`
- native `Token` instances of the Stanza doc receive the NER tags
- native `Span` instances of the Stanza doc receive the contiguous merged NER tags

In [5]:
for article in mybook:
    article._enrich_stanzadoc()
    #break

We search now for a pattern in the structure of the article-body :

... NC_Spatial ... NP_Spatial ... NP_Spatial ...

Strings are stored in the corresponding attributes : nc1, np1, np2


In [6]:
# loading stopwords from Solr
with open('solr_stopwords.txt', 'r') as f:
    stopwords = f.read().split('\n') 

In [7]:
for article in mybook:
    article._search_spatial_pattern(stopwords)
    #break

The GE phrases are also normalized during the search for the pattern :
- `ancienne ville` to `ville`
- `royaume de France` to `france`
- `St. Etienne` to `saint etienne`
- 
The normalized strings are stored in `nps` (Proper Noun) and `ncs` (Common Noun) attributes :


In [22]:
for np in grenoble.nps:
    print(np.text, ' | ', np.norm_text)

France  |  france
Dauphiné  |  dauphine
Vienne  |  vienne
Grenoble  |  grenoble
l'Isere  |  isere
Chambéri  |  chamberi
Turin  |  turin
Vienne  |  vienne
Paris  |  paris
Gratianopolis  |  gratianopolis
Cularo  |  cularo
Grenoble  |  grenoble
Grenoble  |  grenoble
Dauphiné  |  dauphine
Angleterre  |  angleterre
Solbaye  |  solbaye
Ruyter  |  ruyter


In [20]:
for nc in grenoble.ncs:
    print(nc.text, ' | ', nc.norm_text)

ancienne ville  |  ville
capitale  |  capitale
un évêché suffragant  |  eveche
Cette ville  |  ville
cette ville  |  ville
pays  |  pays


### Inspecting the string normalization

In [45]:
np_pairs = {} # pair (string, normalized string)
nc_pairs = {} # pair (string, normalized string)
for article in mybook:
    for np in article.nps:
        np_pairs[np.text] = np.norm_text
    for nc in article.ncs:
        nc_pairs[nc.text] = nc.norm_text

In [46]:
# print 20 random pairs
import random
print('-------------------------------   NP')
for i in range(10):
    print(random.choice(list(np_pairs.items())))
print('-------------------------------   NC')
for i in range(10):
    print(random.choice(list(nc_pairs.items())))

-------------------------------   NP
('Narbata', 'narbata')
('Meuze', 'meuze')
('Autgard', 'autgard')
('Sparte', 'sparte')
('Ganesborough', 'ganesborough')
('Amadabad', 'amadabad')
('Aquitaines', 'aquitaines')
('Lestrygoniae rupes', 'lestrygoniae')
('Zambra', 'zambra')
('Willonghby', 'willonghby')
-------------------------------   NC
('Sa cathédrale', 'cathedrale')
('le promontoire', 'promontoire')
('la contrée', 'contree')
('les cavernes', 'cavernes')
('long promontoire', 'promontoire')
('ville vieille', 'ville')
('états', 'etats')
('un palais épiscopal', 'palais')
('un évêché', 'eveche')
('les terres', 'terres')


How many articles do not have any NPs following the 1st NC ?

In [47]:
len([article for article in mybook if article.nc1_ and not article.np1_ and not article.np2_])

169

# Absolute count of NP-Spatial
(Geographic Entities that are NP with a proper Noun)

In [48]:
mybook._make_counts()

In [49]:
mybook.nps_counter.most_common(10)

[('france', 2108),
 ('italie', 1575),
 ('allemagne', 1379),
 ('asie', 928),
 ('afrique', 890),
 ('espagne', 841),
 ('angleterre', 688),
 ('paris', 514),
 ('sicile', 423),
 ('rome', 419)]

In [50]:
mybook.ncs_counter.most_common(10)

[('ville', 12973),
 ('riviere', 3427),
 ('ile', 2957),
 ('pays', 2867),
 ('mer', 2514),
 ('province', 2334),
 ('royaume', 1913),
 ('capitale', 1763),
 ('cote', 1328),
 ('fleuve', 1178)]

# Re order NP1 and NP2
When defining the HEADPHRASE, the authors can mention NP1 and NP2 in the article-body. They will help us querying our Knowledge Base by re-framing the search : we will limit the search to an area defined by NP1, or NP2, or the two.

Hypothesis : the more a NP is quoted in the corpus, the higher the probabilities are that it's a big geographical entity, possibly containing the other NP mentionned int the corpus.

We therefore characterize each NP by its absolute count in the corpus, and redefine these 2 attributes :
- NP1* is the least quoted entity between NP1 and NP2
- NP2* is the most quoted entity between NP1 and NP2

In [31]:
for article in mybook:
    if not article.np2_:
        np1_star = article.np1_
        np2_star = None
    elif mybook.nps_counter[article.np1_] < mybook.nps_counter[article.np2_] :
        np1_star = article.np1_
        np2_star = article.np2_
    else:
        np1_star = article.np2_
        np2_star = article.np1_
    
    article.np1_star = np1_star
    article.np2_star = np2_star

### Inspecting the reordering

In [35]:
for article in mybook._sample(20):
    print(article.np1_star, article.np2_star)

lothian ecosse meridionale
gaster suisse
iraque arabiquel chaldee
tirol allemagne
japon None
paris france
treves allemagne
jascartes sihon
brabant hollandois pays bas
cetina dalmatie
samnis samnium
sarmatie europe
croatie save
ruse mingrelie
carnie italie
provence france
thessalonique macedoine
istrie rome
mogol indes
None None


# Save the Book

In [33]:
mybook.description  = 'version with POS tags, enriched Stanza Spans with NER tags'
mybook.description += '\nand spatial patterns, and with reordered NP1* and NP2*'
with open(datapath+'geobook_26082024.pkl', 'wb') as f:
    pickle.dump(mybook, f)
mybook

Book with 15510 articles
version with POS tags, enriched Stanza Spans with NER tags and spatial patterns, and with reordered NP1* and NP2*
Attributes :
dict_keys(['volume', 'numero', 'authors', 'text', 'hash', 'artfl', 'gold_coords', 'enccre', 'article_id', 'gold_qid', 'parsed', 'ner', 'nc1', 'nc1_', 'np1', 'np1_', 'np2', 'np2_', 'ncs', 'nps', 'headphrase', 'np1_star', 'np2_star'])